In [ ]:
import numpy as np
import pandas as pd
import keras 

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow 
from mlflow.models import infer_signature 



ImportError: cannot import name 'type' from 'hyperopt' (/home/kai/repo/lab8/mlflow_101/.venv/lib/python3.10/site-packages/hyperopt/__init__.py)

In [9]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv",
    sep=",",
)

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [10]:
train, test = train_test_split(data, test_size=0.3, random_state=50)

In [11]:
train_x = train.drop(['quality'], axis=1).values
train_y = train['quality'].values.ravel()

test_x = test.drop(['quality'], axis=1).values
test_y = test['quality'].values.ravel()

In [12]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=56)
signature = infer_signature(train_x, train_y)

In [16]:
from itsdangerous import SignatureExpired


def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    
    model = keras.Sequential([
        keras.Input(shape=(train_x.shape[1],)),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params['lr'],
            momentum=params['momentum']
        ),
        loss='mean_squared_error',
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=epochs, batch_size=64)
        
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]
        
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        mlflow.tensorflow.log_model(model, "model", signature=signature)
    return {"loss":eval_rmse, "status": STATUS_OK, "model": model}


In [17]:
def objective(params):
    result = train_model(
        params=params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    
    return result

In [18]:
space = {
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0,1.0)
}

In [21]:
mlflow.set_experiment("/wine_quality")
with mlflow.start_run():
    trials=Trials()
    best= fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]
    
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], 'model', signature=signature)
    
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")
    

2025/05/25 20:21:35 INFO mlflow.tracking.fluent: Experiment with name '/wine_quality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

E0000 00:00:1748184695.799511   43229 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1748184695.801645   43229 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/3                                            

 1/43 ━━━━━━━━━━━━━━━━━━━━ 19s 453ms/step - loss: 37.7571 - root_mean_squared_error: 6.1447
28/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.4802 - root_mean_squared_error: 6.0396   
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 35.9446 - root_mean_squared_error: 5.9949 - val_loss: 30.9835 - val_root_mean_squared_error: 5.5663

Epoch 2/3                                            

 1/43 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 33.2124 - root_mean_squared_error: 5.7630
32/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.5802 - root_mean_squared_error: 5.4377 
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.9891 - root_mean_squared_error: 5.3826 - val_loss: 23.5825 - val_root_mean_squared_error: 4.8562

Epoch 3/3                                            

 1/43 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 23.8852 - root_mean_squared_error: 4.8872
35/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22.1840 - root_mean_squared_error: 4.7